https://satijalab.org/seurat/archive/v3.1/atacseq_integration_vignette.html![image.png](attachment:image.png)

https://satijalab.org/seurat/archive/v3.0/atacseq_integration_vignette.html
Note: this data isn't matched.
Data from https://satijalab.org/signac/articles/pbmc_multiomic.html

In [1]:
library(Seurat)
library(ggplot2)
library(patchwork)

In [2]:
data <- readRDS("../../data/pbmc/data_list.rds")
peaks <- data$Peaks
genes = data[['Gene Expression']]

In [3]:
atac <- readRDS("../../Seurat3/pbmc/atac.rds")

In [5]:
activity.matrix = atac[['ACTIVITY']]

In [6]:
rna_count = data[["Gene Expression"]]
shared_genes = intersect(rownames(rna_count), rownames(activity.matrix))
colnames(rna_count) = paste0("rna.", colnames(rna_count))
colnames(activity.matrix) = paste0("atac.", colnames(activity.matrix))

In [8]:
cat_count_matrix = cbind(rna_count[shared_genes,], activity.matrix[shared_genes,])

In [11]:
str(cat_count_matrix)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:48255478] 14 19 34 42 43 49 58 60 70 78 ...
  ..@ p       : int [1:22649] 0 2014 3201 4992 5472 6892 7715 9625 10661 12582 ...
  ..@ Dim     : int [1:2] 11045 22648
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:11045] "AP006222.2" "LINC01128" "AL645608.2" "SAMD11" ...
  .. ..$ : chr [1:22648] "rna.AAACAGCCAAGGAATC-1" "rna.AAACAGCCAATCCCTT-1" "rna.AAACAGCCAATGCGCT-1" "rna.AAACAGCCACACTAAT-1" ...
  ..@ x       : num [1:48255478] 1 1 1 1 1 1 1 6 3 3 ...
  ..@ factors : list()


In [10]:
cat_count_matrix <- cat_count_matrix[-which(apply(cat_count_matrix, 1, var) == 0),]

In [ ]:
cat_mat.pca = prcomp(t(as.matrix(cat_count_matrix)), center = T, scale. = T, rank. = 20)

## Incorporate cell type

In [1]:
embedding <- read.table("../../Raw/pbmc/integrated_embedding.txt")

In [2]:
meta <- read.table("../../data/pbmc/cell_type_h5ad.txt", stringsAsFactors = F, head = T)

In [3]:
str(meta)

'data.frame':	23818 obs. of  2 variables:
 $ bc     : chr  "TCCTTAGTCATTATGG-1" "GGCAAGCCAACATAAG-1" "GTCTCACTCATGAAGG-1" "CGTTAAATCTCCTCTT-1" ...
 $ cluster: int  11 9 3 4 8 12 10 6 17 17 ...


In [4]:
meta.ft = c(meta$cluster, meta$cluster)
names(meta.ft) = c(paste0("rna.", meta$bc), paste0("atac.", meta$bc))

In [5]:
#embedding = as.data.frame(cat_mat.pca$x[,1:20])
#stopifnot(rownames(embedding) != embedding$bc)
embedding$cell_type = factor(meta.ft[rownames(embedding)])

In [6]:
embedding$batch = as.numeric(factor(sapply(strsplit(rownames(embedding), split = ".", fixed = T), function(s) paste0(s[2:length(s)], collapse = "."))))

In [7]:
str(embedding)

'data.frame':	23818 obs. of  22 variables:
 $ PC1      : num  -27.9 -31.9 -28.3 -34.6 -31.3 ...
 $ PC2      : num  9.129 0.919 6.193 -5.109 2.952 ...
 $ PC3      : num  -28.53 -2.12 -17.43 11.19 -10.29 ...
 $ PC4      : num  -27.13 -7.78 -20.31 -2.88 -16.93 ...
 $ PC5      : num  13.0748 3.1967 8.2698 0.0484 6.661 ...
 $ PC6      : num  0.3314 0.2641 0.0985 -0.8097 0.1685 ...
 $ PC7      : num  -3.539 -1.485 -2.564 0.243 -0.579 ...
 $ PC8      : num  -12.47 -3.09 -7.65 0.41 -4.05 ...
 $ PC9      : num  -1.6721 0.5126 -0.9143 1.3738 -0.0434 ...
 $ PC10     : num  6.945 1.906 3.841 0.558 3.699 ...
 $ PC11     : num  -0.371 0.11 1.398 0.98 1.457 ...
 $ PC12     : num  2.344 0.145 1.237 -0.112 1.137 ...
 $ PC13     : num  8.226 -0.634 5.157 0.449 4.721 ...
 $ PC14     : num  2.688 -0.857 2.065 1.194 4.305 ...
 $ PC15     : num  0.1992 -0.4413 0.1469 -0.0302 0.2513 ...
 $ PC16     : num  0.2631 -0.1527 -0.43 -0.0453 -0.5545 ...
 $ PC17     : num  2.36978 0.58112 1.69392 0.00319 0.702 ...
 $

In [8]:
write.table(embedding, "../../Raw/pbmc/integrated_embedding.txt", quote = F, sep = "\t", row.names = T, col.names = T)